## Balancing a Dataset with Downsampling
- Imagine we have a dataset for a binary classification task where the class labels are imbalanced, and we want to downsample the majority class to balance the dataset.

In [17]:
import pandas as pd
from sklearn.utils import resample

# Sample Dataset
df = pd.DataFrame({
    'Age' : [22, 25, 27, 28, 30, 35, 40, 45, 50, 55, 60, 65, 70],
    'Income' : [2000, 2500, 2700, 3200, 3500, 3800, 4000, 4200, 4300, 4500, 5000, 5500, 6000], 
    'Class' : ['High', 'Low', 'Low', 'High', 'High', 'Low', 'High', 'High', 'Low', 'Low', 'High', 'High',  'Low']
})

In [18]:
df

,Age,Income,Class
0,22,2000,High
1,25,2500,Low
2,27,2700,Low
3,28,3200,High
4,30,3500,High
5,35,3800,Low
6,40,4000,High
7,45,4200,High
8,50,4300,Low
9,55,4500,Low


In [19]:
# Seperate High and Low classes
df_high = df[df['Class'] == 'High']
df_low = df[df['Class'] == 'Low']

In [20]:
# Downsample High class
df_high_downsampled = resample(df_high, replace=False, n_samples=len(df_low), random_state=42)

In [21]:
# Combine downsampled High with Low class
df_balanced = pd.concat([df_high_downsampled, df_low])

In [22]:
print(df_balanced['Class'].value_counts())

High    6
Low     6
Name: Class, dtype: int64


In [23]:
df_balanced

,Age,Income,Class
0,22,2000,High
3,28,3200,High
10,60,5000,High
4,30,3500,High
7,45,4200,High
6,40,4000,High
1,25,2500,Low
2,27,2700,Low
5,35,3800,Low
8,50,4300,Low


## Upsampling the Minority Class
- Lets use a dataset with a binary classification task where the Minority class has fewer instances than the Majority class, and we'll perform upsampling on the minority class.

In [24]:
import pandas as pd
from sklearn.utils import resample

# Sample Dataset
df = pd.DataFrame({
    'Age' : [22, 25, 27, 28, 30, 35, 40, 45, 50, 55, 60, 65, 70],
    'Income' : [2000, 2500, 2700, 3200, 3500, 3800, 4000, 4200, 4300, 4500, 5000, 5500, 6000], 
    'Class' : ['Minority', 'Majority', 'Majority', 'Majority', 'Majority', 'Minority', 'Minority', 'Minority', 'Majority', 'Majority', 'Majority', 'Majority',  'Majority']
})

In [25]:
df

,Age,Income,Class
0,22,2000,Minority
1,25,2500,Majority
2,27,2700,Majority
3,28,3200,Majority
4,30,3500,Majority
5,35,3800,Minority
6,40,4000,Minority
7,45,4200,Minority
8,50,4300,Majority
9,55,4500,Majority


In [26]:
# Seperate majority and minority classes
df_majority = df[df['Class'] == 'Majority']
df_minority = df[df['Class'] == 'Minority']

In [27]:
# Upsample Minority class
df_minority_upsampled = resample(df_minority, replace=True, n_samples=len(df_majority), random_state=42)

In [28]:
# Combine upampled Minority with Majority class
df_balanced = pd.concat([df_majority, df_minority_upsampled])

In [29]:
print(df_balanced['Class'].value_counts())

Majority    9
Minority    9
Name: Class, dtype: int64


In [30]:
df_balanced

,Age,Income,Class
1,25,2500,Majority
2,27,2700,Majority
3,28,3200,Majority
4,30,3500,Majority
8,50,4300,Majority
9,55,4500,Majority
10,60,5000,Majority
11,65,5500,Majority
12,70,6000,Majority
6,40,4000,Minority


## SMOTE (Synthetic Minority Over-sampling Technique)
- SMOTE generates synthetic examples rather than simply duplicating existing ones.

#### Steps : 
1. SMOTE to generate synthetic samples instead of duplicating existing ones.
2. Convert categorical class labels into numeric form for SMOTE to work.
3. Apply SMOTE to balance the dataset.
4. Convert back to original categorical labels.
5. Combine the resampled data into a final balanced dataset

In [31]:
pip install imbalanced-learn

Note: you may need to restart the kernel to use updated packages.


In [32]:
import pandas as pd
from imblearn.over_sampling import SMOTE

# Sample Dataset
df = pd.DataFrame({
    'Age' : [22, 25, 27, 28, 30, 35, 40, 45, 50, 55, 60, 65, 70],
    'Income' : [2000, 2500, 2700, 3200, 3500, 3800, 4000, 4200, 4300, 4500, 5000, 5500, 6000], 
    'Class' : ['Minority', 'Majority', 'Majority', 'Majority', 'Majority', 
               'Minority', 'Minority', 'Minority', 'Majority', 'Majority', 
               'Majority', 'Majority',  'Majority']
})

# Step 1 : Convert categorical labels to numeric values
df['Class'] = df['Class'].map({'Majority' : 0, 'Minority' : 1})

# Step 2 : Split features (X) and target variable (y)
X = df[['Age','Income']]
y = df['Class']

# Step 3 : Apply SMOTE with k_neighbors = 3 (reducing from default 5)
smote = SMOTE(sampling_strategy='auto', random_state=42, k_neighbors=3)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 4 : Convert numeric labels back to categorical
y_resampled = y_resampled.map({0 : 'Majority', 1 : 'Minority'})

# Step 5 : Combine the resampled data
df_balanced = pd.concat([pd.DataFrame(X_resampled, columns = ['Age', 'Income']), pd.DataFrame(y_resampled, columns = ['Class'])], axis = 1)

# Step 6: Print class distribution
print(df_balanced['Class'].value_counts())
                         
# Step 7: Display the upsampled dataset
print(df_balanced)

Minority    9
Majority    9
Name: Class, dtype: int64
    Age  Income     Class
0    22    2000  Minority
1    25    2500  Majority
2    27    2700  Majority
3    28    3200  Majority
4    30    3500  Majority
5    35    3800  Minority
6    40    4000  Minority
7    45    4200  Minority
8    50    4300  Majority
9    55    4500  Majority
10   60    5000  Majority
11   65    5500  Majority
12   70    6000  Majority
13   40    4031  Minority
14   35    3831  Minority
15   44    4176  Minority
16   35    3826  Minority
17   41    4040  Minority
